In [5]:
import numpy as np
import matplotlib.pyplot as plt
from entities import *
import helpers
import plotting
%matplotlib qt

In [6]:
import file_operations as fo
equipment, obstacles = fo.read_configuration_csv("configuration.txt")

from obstacle_casting import cast_points_to_shape
# Apply obstacles blockage to signal boundary points
for i, eq in enumerate(equipment):
    x, y = eq.polygon.exterior.coords.xy
    for obstacle in obstacles:
        cast_points_to_shape(x, y, eq.source_point, obstacle.polygon)
    equipment[i] = Entity(eq.source_point.xy, x, y, eq.entity_type)

In [7]:
import pickle

with open('equipment.pkl', 'wb') as f:
    pickle.dump(equipment[7], f)

In [8]:
def run_through_configuration():
    from obstacle_casting import cast_points_to_shape

    # Apply obstacles blockage to signal boundary points
    for i, eq in enumerate(equipment):
        x, y = eq.polygon.exterior.coords.xy
        for obstacle in obstacles:
            cast_points_to_shape(x, y, eq.source_point, obstacle.polygon)
        equipment[i] = Entity(eq.source_point.xy, x, y, eq.entity_type)

    # plotting.plot_map(equipment, obstacles)

    G = helpers.create_directional_graph(equipment)

    def on_obstacle_click(event):
        for obstacle in obstacles:
            if obstacle.polygon.contains(Point(event.mouseevent.xdata, event.mouseevent.ydata)):
                print("Obstacle:", obstacle.entity_type)
                obstacles.remove(obstacle)
                # run_through_configuration()
                break
    
    try:
        dst, path = helpers.nx.single_source_dijkstra(
            G, source=equipment[0], target=equipment[1])
        
        centres = [eq.source_point for eq in path]
        plt.clf()
        
        plotting.plot_map(equipment, obstacles, selected_eqs=path)
        
        plt.gcf().canvas.mpl_connect('pick_event', on_obstacle_click)
        carr = np.array(centres)
        # Plot arrows between points
        for i in range(len(carr) - 1):
            x1, y1 = carr[i].coords[0]
            x2, y2 = carr[i + 1].coords[0]
            plt.arrow(x1, y1, (x2 - x1), (y2 - y1), head_width=0.1,
                    head_length=0.1, fc='r', ec='r')
        plt.grid(False)
    except helpers.nx.NetworkXNoPath as e:
        # plt.clf()
        print(e)

In [9]:
# print("Equipment:")
# _ = list(map(print, [e.entity_type for e in equipment]))
# print("\n")
# print("Obstacles:")
# _ = list(map(print, [o.entity_type for o in obstacles]))

In [10]:
# plotting.plot_graph(G)

In [11]:
run_through_configuration()

Checking edge from EntityType.USER at POINT (25 25) to EntityType.YAGI_ANTENNA at POINT (40 25)
Shortest path in polygon
Triangulated polygon POLYGON ((25 50, 25.87735965246312 49.9846000576401, 26.753638401880092 49.93841920321839, 27.627756676872337 49.86151433133454, 28.49863756775581 49.75398018843612, 29.365208153288755 49.61594925609129, 30.226400822505735 49.44759158777222, 31.081154590009405 49.249114599349966, 31.928416403099654 49.02076281355902, 32.76714243912966 48.762817558746505, 33.59629939149056 48.47559662227691, 34.41486574264047 48.159453859019536, 35.22183302260936 47.81477875540091, 36.01620705142939 47.44199594955933, 36.79700916396009 47.0415647081926, 37.56327741559925 46.61397836074363, 38.31406776739428 46.15976369162077, 39.04845524909382 45.67948029120191, 39.76553509870683 45.17371986642168, 40.46442387716518 44.64310551179124, 41.144260556716546 44.088290941748724, 41.80420758170659 43.50995968528605, 42.44345190044366 42.908824243844386, 43.06120596687478